# 重新定义大模型和智能体

## IO：基于标准输入输出的流式调用

In [ ]:
from textlong.io import TextBlock, stream_log

In [ ]:
def hello():
    yield TextBlock("info", "hello textlong")
    yield TextBlock("chunk", "Textlong")
    yield TextBlock("chunk", "很")
    yield TextBlock("chunk", "棒！")

stream_log(hello)

## 调用大模型

### 大模型能力：通义千问

In [ ]:
from textlong.llm.dashscope import qwen
from textlong.io import stream_log

stream_log(
    qwen,
    [{"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}],
    model="qwen2-1.5b-instruct"
)

### fake_llm: 模拟大模型调用过程

这可以观察提示语等内部结构。

In [ ]:
from textlong.llm import fake_llm
from textlong.io import stream_log

stream_log(
    fake_llm,
    [
        {"role": "system", "content": "你是一个写作能手。"},
        {"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}
    ],
    model="qwen2-1.5b-instruct"
)

### 多轮对话

使用`messages`管理多轮对话的记忆。<br>
你也可以在`chat`函数中通过参数`k`限制希望保留的对话轮次，但这不影响`messages`记录完整的对话过程。

In [ ]:
from textlong.llm import chat, fake_llm
from textlong.llm.dashscope import qwen
messages = []

In [ ]:
chat(qwen, "你能帮我写一首关于兔子做梦的四句儿歌?", messages)

In [ ]:
chat(qwen, "换成小羊吧", messages)

In [ ]:
messages

In [ ]:
chat(qwen, "改为在蓝天上", messages, k=1)

In [ ]:
chat(fake_llm, "改为在草地上", messages, k=10)

In [ ]:
messages

### 在大模型中支持回调工具

仍然采纳`langchain`来定义工具，这可以大大简化回调工具的管理。

涉及定义工具的技巧主要包括：
- 使用 tool 装饰函数将新函数定义为工具类型
- 使用 StructTool 转换已有的函数为工具类型
- 使用 BaseModel 补充工具的参数定义

采用以上方法定义好工具，就可以使用 convert_to_openai_tool 将其转换为官方要求的 JSON 结构（确实会比手写节省很多精力）。

In [ ]:
from textlong.llm.tools import tool, convert_to_openai_tool, 
from textlong.llm.dashscope import qwen
from textlong.io import stream_log
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

stream_log(qwen, "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)])

### 在多轮对话中支持回调工具

In [ ]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm.dashscope import qwen
from textlong.llm import chat
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

messages = []
chat(qwen, "今天广州天气如何啊", messages, tools=[convert_to_openai_tool(get_current_weather)], toolkits=[get_current_weather])

In [ ]:
chat(qwen, "这是怎么得出来的？", messages, tools=[convert_to_openai_tool(get_current_weather)], toolkits=[get_current_weather])

In [ ]:
chat(qwen, "再看看上海", messages, tools=[convert_to_openai_tool(get_current_weather)], toolkits=[get_current_weather])

In [ ]:
messages

## 写作提示语

In [ ]:
from textlong.hub import load_chat_template, load_resource_chat_template

### 加载对话模板的资源

指定资源库中的模板ID即可加载模板。

In [ ]:
load_chat_template("IDEA")

In [ ]:
load_chat_template("OUTLINE")

### 查看构造出的提示语模板

In [ ]:
print(load_chat_template("OUTLINE").format(task="请帮我写一首歌"))

### 单轮写作

In [8]:
from textlong.llm import chat, write, fake_llm
from textlong.llm.dashscope import qwen
messages = []

In [9]:
write(qwen, "IDEA", {"task": "帮我写一首关于兔子的四句儿歌"}, messages)

小白兔，蹦蹦跳，  
耳朵长，尾巴小。  
爱吃萝卜和青菜，  
快乐生活在林间。



'小白兔，蹦蹦跳，  \n耳朵长，尾巴小。  \n爱吃萝卜和青菜，  \n快乐生活在林间。'

In [10]:
write(qwen, "IDEA", {"task": "帮我写一首关于小马的四句儿歌"}, messages)

小马儿小马儿, 欢快在草原跑,
四蹄轻盈如飞, 毛发闪亮真妙。
尾巴摇摇乐悠悠, 叫声咿呀真娇俏,
朋友遍地都是, 快乐生活多美好。



'小马儿小马儿, 欢快在草原跑,\n四蹄轻盈如飞, 毛发闪亮真妙。\n尾巴摇摇乐悠悠, 叫声咿呀真娇俏,\n朋友遍地都是, 快乐生活多美好。'

In [11]:
messages

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n帮我写一首关于小马的四句儿歌\n'},
 {'role': 'user', 'content': '请你开始'},
 {'role': 'assistant',
  'content': '小马儿小马儿, 欢快在草原跑,\n四蹄轻盈如飞, 毛发闪亮真妙。\n尾巴摇摇乐悠悠, 叫声咿呀真娇俏,\n朋友遍地都是, 快乐生活多美好。'}]

### 单轮写作 + 多轮对话

In [12]:
chat(qwen, "改为8句吧", messages)

小马儿小马儿, 草原上蹦蹦跳,
四蹄蹬开晨露, 迎着阳光笑。
鬃毛飘飘似彩云, 眼中星光闪耀,
穿过青青草地, 快乐歌声飘。

彩虹下它起舞, 尾巴摇出欢乐曲,
小河旁饮水清, 波光里映俏影。
朋友小兔小鹿, 一起玩耍不孤寂,
小马儿的天地, 满是爱与奇迹。



'小马儿小马儿, 草原上蹦蹦跳,\n四蹄蹬开晨露, 迎着阳光笑。\n鬃毛飘飘似彩云, 眼中星光闪耀,\n穿过青青草地, 快乐歌声飘。\n\n彩虹下它起舞, 尾巴摇出欢乐曲,\n小河旁饮水清, 波光里映俏影。\n朋友小兔小鹿, 一起玩耍不孤寂,\n小马儿的天地, 满是爱与奇迹。'

In [ ]:
messages

In [ ]:
chat(qwen, "类似西方的诗歌那样风格", messages, k=3, model="qwen2-1.5b-instruct")

In [ ]:
messages